In [28]:
import findspark
findspark.init()


import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [29]:
#load Green Taxi Data

df_green = spark.read.parquet('data/pq/green/*/*')

In [30]:
df_green.registerTempTable('green')

/home/Ethan/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [31]:
# Use triple quotes for multi-line strings
df_result = spark.sql( """
 SELECT
    -- Reveneue grouping 
       date_trunc("hour", lpep_pickup_datetime)  as hour, 
    PULocationID as zone,


    -- Revenue calculation 

    SUM(total_amount) AS amount,
    count(1) as number_records

    FROM green
    where lpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY 1,2
     order by 1,2
    
""")

In [32]:
df_result.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 769.7299999999996|            45|
|2020-01-01 00:00:00|  17|195.03000000000003|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25| 531.0000000000002|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|317.27000000000004|            11|
|2020-01-01 00:00:00|  35|            129.96|             5|
|2020-01-01 00:00:00|  36|295.34000000000003|            11|
|2020-01-01 00:00:00|  37|            175.67|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [33]:
df_green_revenue = df_result

df_green_revenue \
.repartition(20) \
.write.parquet('data/report/revenue/green',mode="overwrite")

In [34]:
#load Yellow Taxi Data

df_yellow = spark.read.parquet('data/pq/yellow/*/*')

df_yellow.registerTempTable('yellow')

In [35]:
# Use triple quotes for multi-line strings
df_yellow_result = spark.sql( """
 SELECT
    -- Reveneue grouping 
       date_trunc("hour", tpep_pickup_datetime)  as hour, 
    PULocationID as zone,


    -- Revenue calculation 

    SUM(total_amount) AS amount,
    count(1) as number_records

    FROM yellow
    where tpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY 1,2
    order by 1,2
""")

df_yellow_result.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   3|              25.0|             1|
|2020-01-01 00:00:00|   4|1004.3000000000002|            57|
|2020-01-01 00:00:00|   7| 455.1700000000001|            38|
|2020-01-01 00:00:00|  10|             42.41|             2|
|2020-01-01 00:00:00|  12|             107.0|             6|
|2020-01-01 00:00:00|  13|1214.8000000000002|            56|
|2020-01-01 00:00:00|  14|               8.8|             1|
|2020-01-01 00:00:00|  15|             34.09|             1|
|2020-01-01 00:00:00|  17|220.20999999999998|             8|
|2020-01-01 00:00:00|  18|               5.8|             1|
|2020-01-01 00:00:00|  24| 754.9500000000002|            45|
|2020-01-01 00:00:00|  25|            324.35|            16|
|2020-01-01 00:00:00|  32|              18.0|             1|
|2020-01-01 00:00:00|  3

In [36]:
df_yellow_result \
.repartition(20) \
.write.parquet('data/report/revenue/yellow',mode="overwrite")

In [46]:
#spark.stop()

df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

df_green_revenue_temp = df_green_revenue.withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records') 
df_yellow_result_temp = df_yellow_revenue.withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records') 

In [47]:
df_join = df_green_revenue_temp.join(df_yellow_result_temp, on=['hour','zone'], how='outer')

In [48]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|
|2020-01-01 00:00:00|   7| 769.7299999999996|                  45| 455.1700000000001|                   38|
|2020-01-01 00:00:00|  12|              null|                null|             107.0|                    6|
|2020-01-01 00:00:00|  37|            175.67|                   6|161.60999999999999|                    7|
|2020-01-01 00:00:00|  40|168.97999999999996|                   8|             89.97|                    5|
|2020-01-01 00:00:00|  45|  

In [49]:
df_join.write.parquet('data/report/revenue/total',mode='overwrite')

In [51]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2024-02-28 13:07:32--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 16.182.69.64, 52.217.171.128, 52.217.201.40, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|16.182.69.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0s      

2024-02-28 13:07:32 (153 MB/s) - ‘taxi+_zone_lookup.csv’ saved [12322/12322]



In [63]:
df_zones = df = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')
df_zones.write.parquet('zones/',mode='overwrite')

In [64]:
df_zones = spark.read.parquet('zones/')
df_join = spark.read.parquet('data/report/revenue/total')

In [65]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [66]:
df_result = df_join.join(df_zones,df_join.zone == df_zones.LocationID)

In [70]:
df_result.drop('LocationID','zone').write.parquet('tmp/revenue-zones')